# dask-sql

### A Python SQL query engine built on top of Dask

* [Website](https://nils-braun.github.io/dask-sql/)
* [Docs](https://dask-sql.readthedocs.io/en/latest/)
* [GitHub](https://github.com/nils-braun/dask-sql)

* **Combine the power of Python and SQL**: load your data with Python, transform it with SQL, enhance it with Python and query it with SQL - or the other way round. With dask-sql you can mix the well known Python dataframe API of pandas and Dask with common SQL operations, to process your data in exactly the way that is easiest for you.

* **Infinite Scaling**: using the power of the great Dask ecosystem, your computations can scale as you need it - from your laptop to your super cluster - without changing any line of SQL code. From k8s to cloud deployments, from batch systems to YARN - if Dask supports it, so will dask-sql.

* **Your data - your queries**: Use Python user-defined functions (UDFs) in SQL without any performance drawback and extend your SQL queries with the large number of Python libraries, e.g. machine learning, different complicated input formats, complex statistics.

* **Easy to install and maintain**: dask-sql is just a pip/conda install away (or a docker run if you prefer). No need for complicated cluster setups - dask-sql will run out of the box on your machine and can be easily connected to your computing cluster.

* **Use SQL from wherever you like**: dask-sql integrates with your jupyter notebook, your normal Python module or can be used as a standalone SQL server from any BI tool. It even integrates natively with Apache Hue.

## Preparations

1. Install `dask-sql` via `conda` or `pip`.

    ```bash
conda install dask-sql
mamba install dask-sql
pip install dask-sql
    ```

2. Prepare a `dask-sql` context to store the table meta-information.

    ```python
from dask_sql import Context    
c = Context()
    ```
    
That's it! All other steps are optional:

3. Create a dask cluster and connect to it.
3. Enable the `%%sql` jupyter magic.

In [ ]:
from collections import OrderedDict

from dask_sql import Context
from dask.distributed import Client
import dask.dataframe as dd

# Use a local Dask cluster 
client = Client()
c = Context()

In [ ]:
c.ipython_magic()

In [ ]:
client

### Query via Python Call

In [ ]:
c.sql("SELECT 1 + 1")

In [ ]:
c.sql("SELECT 1 + 1").compute()

In [ ]:
c.sql("SELECT 1 + 1", return_futures=False)

### Query via Notebook Integration

In [ ]:
%%sql
SELECT 1 + 1

### Query via SQL Server 

```python
c.run_server(blocking=False)
c.stop_server()
```

## Read in the Data

### Download the data for the gods of the live demo

```bash
mkdir -p data
wget https://s3.amazonaws.com/bsql/data/air_transport/airlines.csv -P data/
wget https://s3.amazonaws.com/bsql/data/air_transport/airports.csv -P data/
wget https://s3.amazonaws.com/bsql/data/air_transport/flight_ontime_2020-01.parquet -P data/

```

In [ ]:
! ls -lh data/

### Reading Data 1: Register a Dask/Pandas Data Frame

We read it via a `Dask` API call and register it at the context

In [ ]:
flight_data_path = './data/flight_ontime_2020-01.parquet'

air_transport = dd.read_parquet(flight_data_path)
air_transport["FL_DATE"] = air_transport["FL_DATE"].astype("datetime64[ns]")

c.create_table('air_transport', air_transport, persist=True)

In [ ]:
c.sql("SELECT * FROM air_transport LIMIT 5").compute()

In [ ]:
%%sql
SELECT * FROM air_transport LIMIT 5

In [ ]:
c.sql("SELECT COUNT(*) AS CNT FROM air_transport").compute()

### Reading Data 2: Read directly with dask-sql

We read it via the dask-sql `create_table` function

In [ ]:
airports_path = "./data/airports.csv" 

airports_dtypes = OrderedDict([
    ("Airport ID", "int64"),
    ("Name", "str"),
    ("City", "str"),
    ("Country", "str"),
    ("IATA", "str"),
    ("ICAO", "str"),
    ("Latitude", "float64"),
    ("Longitude", "float64"),
    ("Altitude", "int64"), 
    ("Unused", "str"), 
    ("Timezone", "str"),
    ("DST", "str"),
    ("Type", "str"),
    ("Source", "str"),
])

c.create_table(
    "airports",
    airports_path, 
    error_bad_lines=False,
    names=list(airports_dtypes.keys()), 
    format="csv",
    dtype=airports_dtypes,
    storage_options={'anon': True}
)

In [ ]:
airlines_path = "./data/airlines.csv"

airlines_dtypes = OrderedDict([
    ('Airline ID', 'int64'),
    ('Name', 'str'),
    ('Alias', 'str'),
    ('IATA', 'str'),
    ('ICAO', 'str'),
    ('Callsign', 'str'),
    ('Country', 'str'),
    ('Active', 'str'),
])

c.create_table(
    "airlines",
    airlines_path,
    names=list(airlines_dtypes.keys()), 
    dtype=airlines_dtypes,
    storage_options={'anon': True},
    format="csv",
)

In [ ]:
%%sql
SELECT * FROM airports LIMIT 5

## Queries

### Build a joined dataset

In [ ]:
%%sql

SELECT A.FL_DATE
    , A.OP_UNIQUE_CARRIER
    , B.Name AS CARRIER_NAME
    , A.ORIGIN
    , C.Name AS ORIGIN_NAME
    , C.City AS ORIGIN_CITY
    , A.DEST
    , D.Name AS DEST_NAME
    , D.City AS DEST_CITY
    , DEP_DELAY
    , ARR_DELAY
    , AIR_TIME
    , DISTANCE
FROM air_transport AS A
LEFT OUTER JOIN airlines AS B
    ON A.OP_UNIQUE_CARRIER = B.IATA
LEFT OUTER JOIN airports AS C
    ON A.ORIGIN = C.IATA
LEFT OUTER JOIN airports AS D
    ON A.DEST = D.IATA
LIMIT 5

### ... and register it for further usage

In [ ]:
%%sql
CREATE OR REPLACE TABLE info AS (
    SELECT A.FL_DATE
        , A.CRS_DEP_TIME
        , A.OP_UNIQUE_CARRIER
        , B.Name AS CARRIER_NAME
        , A.ORIGIN
        , C.Name AS ORIGIN_NAME
        , C.City AS ORIGIN_CITY
        , A.DEST
        , D.Name AS DEST_NAME
        , D.City AS DEST_CITY
        , DEP_DELAY
        , ARR_DELAY
        , AIR_TIME
        , DISTANCE
    FROM air_transport AS A                
    LEFT OUTER JOIN airlines AS B
        ON A.OP_UNIQUE_CARRIER = B.IATA
    LEFT OUTER JOIN airports AS C    
        ON A.ORIGIN = C.IATA
    LEFT OUTER JOIN airports AS D    
        ON A.DEST = D.IATA
    WHERE FL_DATE IS NOT NULL
)

In [ ]:
%%sql
SELECT * FROM info LIMIT 5

In [ ]:
%%sql
SELECT COUNT(*) FROM info

### Number of distinct origins

In [ ]:
%%sql
SELECT COUNT(DISTINCT ORIGIN) FROM info

### Delays per ...

In [ ]:
%%sql
SELECT 
    EXTRACT(DOW FROM FL_DATE) AS "Day of Week",
    AVG(ARR_DELAY) AS "Average Delay" 
FROM info GROUP BY EXTRACT(DOW FROM FL_DATE)

In [ ]:
%%sql
SELECT 
    EXTRACT(DAY FROM FL_DATE) AS "Day of Year",
    AVG(ARR_DELAY) AS "Average Delay" 
FROM info GROUP BY EXTRACT(DAY FROM FL_DATE)
LIMIT 10

In [ ]:
result = c.sql("""
SELECT 
    EXTRACT(DAY FROM FL_DATE) AS "Day of Year",
    AVG(ARR_DELAY) AS "Average Delay" 
FROM info GROUP BY EXTRACT(DAY FROM FL_DATE)
""", return_futures=False)
result.plot(x="Day of Year", y="Average Delay", figsize=(12, 9))

### Ratio of Delayed Flights per Origin

In [ ]:
%%sql
SELECT ORIGIN_NAME AS "Origin"
    , CAST(Delayed_Flights AS FLOAT) / All_Flights AS "Delayed Ratio"
    , Delayed_Flights AS "Delayed Flights"
FROM (
    SELECT ORIGIN_NAME
        , COUNT(*) AS All_Flights
        , COUNT(*) FILTER (WHERE DEP_DELAY > 0) AS Delayed_Flights
    FROM info
    GROUP BY ORIGIN_NAME
)
ORDER BY "Delayed Ratio" DESC
LIMIT 10

### Distribution of Flights per Airline

In [ ]:
%%sql
SELECT CARRIER_NAME
    , COUNT(*)
FROM info
GROUP BY CARRIER_NAME
ORDER BY COUNT(*) DESC
LIMIT 10

### The flights with the largest delay per Airline

In [ ]:
%%sql
WITH AnnotatedData AS (
    SELECT *
        , ROW_NUMBER() OVER (
            PARTITION BY CARRIER_NAME 
            ORDER BY DEP_DELAY DESC NULLS LAST
        ) AS Delay_Order
    FROM info
)
SELECT
    *
FROM AnnotatedData
WHERE Delay_Order = 1
LIMIT 5

### Most Delayed Airlines

In [ ]:
%%sql
SELECT CARRIER_NAME
    , COUNT(*) FILTER (WHERE DEP_DELAY > 0) AS Delayed_Flights
FROM info
GROUP BY CARRIER_NAME
ORDER BY Delayed_Flights DESC
LIMIT 10

### User-Defined Functions

In [ ]:
import numpy as np

def human_readable_dow(dow):
    weekdays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    return dow.apply(lambda i: weekdays[i], meta=(None, "object"))

c.register_function(human_readable_dow, "HR_DOW", 
                    [("dow", np.int64)], np.object_,
                    replace=True)

In [ ]:
%%sql
SELECT HR_DOW(EXTRACT(DOW FROM FL_DATE)) AS "DOW"
    , COUNT(*) AS Flights
FROM info
GROUP BY EXTRACT(DOW FROM FL_DATE)

In [ ]:
result = c.sql("""
SELECT HR_DOW(EXTRACT(DOW FROM FL_DATE)) AS "DOW"
    , COUNT(*) AS Flights
FROM info
GROUP BY EXTRACT(DOW FROM FL_DATE)
""", return_futures=False)
result.plot(x="DOW", y="Flights", kind="bar", 
            title="Flights per Weekday")